<div style="display: flex; align-items: center;">
    <img src="../figures/Mines Geophysics Black Moon Circle Waves 3.3.png" alt="Example Image" width="10%">
    <div style="margin-left: 10px;">
        <h1>GP100 Alumni Map</h1>
        <h2>Data Analysis</h2>
    </div>
</div>

### Loading the Data

In [1]:
# import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import geopandas as gpd

In [2]:
# load data into a dataframe
path = '../data/GP_Alumni_List.csv'
df = pd.read_csv(path)

### Cleaning the Data

In [3]:
# clean the dataframe

#rename StateOrProvince to State
df.rename(columns={'StateOrProvince': 'State'}, inplace=True)

#delete rows if Affiliation is not alumni
df = df[df['Affiliation'] == 'Alumni']

#clean Degree column to include only degree level
validDegrees = ['BSc','MSc','PhD']
df['Degrees'] = df['Degrees'].fillna('').apply(lambda x: [deg.strip() for deg in x.split() if deg.strip() in validDegrees])

#split people with multiple degrees into separate rows
df = df.explode('Degrees')

#replace NaN in country column with United States or Canada based on StateorProvince
usStates = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE',
                                   'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS',
                                   'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS',
                                   'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY',
                                   'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
                                   'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV',
                                   'WI', 'WY','PR']
df['Country'] = df.apply(lambda row: 'United States' if row['State'] in usStates else row['Country'], axis=1)

#remove province from Canada
df['State'] = np.where(df['Country'] == 'Canada', '', df['State'])

df.reset_index(drop=True, inplace=True)
df

,Affiliation,PrefClassYear,Degrees,State,Country
0,Alumni,1983.0,BSc,CO,United States
1,Alumni,1995.0,BSc,CO,United States
2,Alumni,2013.0,MSc,NaN,Bahrain
3,Alumni,1999.0,MSc,NaN,United Kingdom
4,Alumni,1982.0,BSc,OK,United States
...,...,...,...,...,...
2225,Alumni,2012.0,BSc,CO,United States
2226,Alumni,1982.0,BSc,TX,United States
2227,Alumni,1981.0,BSc,UT,United States
2228,Alumni,2019.0,BSc,CO,United States


In [4]:
df.to_csv('../data/GP_Alumni_List_Cleaned.csv', index=False)

## Add Latitudes and Longitudes

In [5]:
# Load the "countries" dataset from geopandas if it's not already loaded
if 'world' not in globals():
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Extract country names and their corresponding centroid coordinates if they're not already extracted
if 'country_coords' not in globals():
    countryCoord = world[['name', 'geometry']].copy()
    countryCoord['Latitude'] = countryCoord['geometry'].centroid.y
    countryCoord['Longitude'] = countryCoord['geometry'].centroid.x
    countryCoord = countryCoord.rename(columns={'name': 'Country'}).drop(columns=['geometry'])

# Merge latitude and longitude coordinates with the original dataframe if they're not already merged
if not all(col in df.columns for col in ['Latitude', 'Longitude']):
    df = pd.merge(df, countryCoord, on='Country', how='left')

C:\Users\renee\AppData\Local\Temp\ipykernel_18036\732390130.py:3: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
C:\Users\renee\AppData\Local\Temp\ipykernel_18036\732390130.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  country_coords['Latitude'] = country_coords['geometry'].centroid.y
C:\Users\renee\AppData\Local\Temp\ipykernel_18036\732390130.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  country_coords['Longitude'] = country_coords['geometry'].centroid.x


In [6]:
#define lat and long for countries not includes in "countries" dataset from geopandas
countryCenters={}

countryCenters['Taiwan, Province of China'] = ['23.6978','120.9605']
countryCenters['Tanzania, United Republic Of'] = ['-6.3690','34.8888']
countryCenters['Bahrain']=['26.0667','50.5577']
countryCenters['NaN']=['']
countryCenters['Singapore']=['1.3521','103.8198']
countryCenters['Korea-South']=['35.9078','127.7669']

In [7]:
# Iterate over the rows of the DataFrame
for index, row in df.iterrows():
    # Check if latitude and longitude are missing and if the country is in countryCenters
    if pd.isnull(row['Latitude']) and pd.isnull(row['Longitude']) and row['Country'] in countryCenters:
        # Update latitude and longitude based on countryCenters
        df.at[index, 'Latitude'] = countryCenters[row['Country']][0]
        df.at[index, 'Longitude'] = countryCenters[row['Country']][1]


C:\Users\renee\AppData\Local\Temp\ipykernel_18036\973064981.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '26.0667' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Latitude'] = countryCenters[row['Country']][0]
C:\Users\renee\AppData\Local\Temp\ipykernel_18036\973064981.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '50.5577' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Longitude'] = countryCenters[row['Country']][1]


In [8]:
# actual states in the data, includes Puerto Rico (PR) and Armed Forces Americas (AA) , 'PR', 'AA'
states =   {'AL': (32.8067, -86.7911),
            'AK': (64.2008, -149.4937),
            'AZ': (34.0489, -111.0937),
            'AR': (34.7998, -92.1997),
            'CA': (36.7783, -119.4179),
            'CO': (39.5501, -105.7821),
            'CT': (41.6032, -73.0877),
            'DE': (38.9108, -75.5277),
            'FL': (27.9944, -81.7603),
            'GA': (33.0406, -83.6431),
            'HI': (19.8968, -155.5828),
            'ID': (44.0682, -114.7420),
            'IL': (40.6331, -89.3985),
            'IN': (39.8647, -86.2604),
            'IA': (42.0115, -93.2105),
            'KS': (38.5266, -96.7265),
            'KY': (37.6690, -84.6514),
            'LA': (31.1695, -91.8678),
            'ME': (45.2538, -69.4455),
            'MD': (39.0458, -76.6413),
            'MA': (42.2596, -71.8083),
            'MI': (44.3467, -85.4102),
            'MN': (46.7296, -94.6859),
            'MS': (32.7416, -89.6787),
            'MO': (38.4561, -92.2884),
            'MT': (46.9219, -110.4544),
            'NE': (41.1254, -98.2681),
            'NV': (38.3135, -117.0554),
            'NH': (43.4525, -71.5639),
            'NJ': (40.2989, -74.5210),
            'NM': (34.8405, -106.2485),
            'NY': (42.1657, -74.9481),
            'NC': (35.6301, -79.8064),
            'ND': (47.5289, -99.7840),
            'OH': (40.3888, -82.7649),
            'OK': (35.5653, -96.9289),
            'OR': (44.5720, -122.0709),
            'PA': (40.5908, -77.2098),
            'RI': (41.6809, -71.5118),
            'SC': (33.8569, -80.9450),
            'SD': (44.2998, -99.4388),
            'TN': (35.7478, -86.6923),
            'TX': (31.0545, -97.5635),
            'UT': (40.1500, -111.8624),
            'VT': (44.0459, -72.7107),
            'VA': (37.7693, -78.1700),
            'WA': (47.4009, -121.4905),
            'WV': (38.4912, -80.9540),
            'WI': (44.2685, -89.6165),
            'WY': (42.7560, -107.3025),
            'PR': (18.2208, -66.5901),
            'AA': (14.4167, -170.7000)}

In [12]:
# Iterate over the rows of the DataFrame
for index, row in df.iterrows():
    # Check if latitude and longitude are missing and if there's a state present
    if pd.isnull(row['Latitude']) and pd.isnull(row['Longitude']) and not pd.isnull(row['State']):
        if row['State'] in states:
            # Update latitude and longitude based on states
            df.at[index, 'Latitude'] = states[row['State']][0]
            df.at[index, 'Longitude'] = states[row['State']][1]
df

,Affiliation,PrefClassYear,Degrees,State,Country,Latitude,Longitude
0,Alumni,1983.0,BSc,CO,United States,39.5501,-105.7821
1,Alumni,1995.0,BSc,CO,United States,39.5501,-105.7821
2,Alumni,2013.0,MSc,NaN,Bahrain,26.0667,50.5577
3,Alumni,1999.0,MSc,NaN,United Kingdom,53.914773,-2.853135
4,Alumni,1982.0,BSc,OK,United States,35.5653,-96.9289
...,...,...,...,...,...,...,...
2225,Alumni,2012.0,BSc,CO,United States,39.5501,-105.7821
2226,Alumni,1982.0,BSc,TX,United States,31.0545,-97.5635
2227,Alumni,1981.0,BSc,UT,United States,40.15,-111.8624
2228,Alumni,2019.0,BSc,CO,United States,39.5501,-105.7821


In [13]:
df.to_csv('../data/GP_Alumni_List_Cleaned.csv', index=False)